
 This example is a EclairJS (JavaScript) implementation of [movie recommending](https://github.com/jadianes/spark-movie-lens/blob/master/notebooks/building-recommender.ipynb).
 #### This example requires the following additional setup
 Download the [movieLens rating dataset](http://grouplens.org/datasets/movielens/)
*[ml-latest-small.zip](http://files.grouplens.org/datasets/movielens/ml-latest-small.zip)
*[ml-latest.zip](http://files.grouplens.org/datasets/movielens/ml-latest.zip) and unzip is a location that is accessible by Spark.
 Add the following options to the eclairjs/kernel.jon SPARK_OPTS
 "SPARK_OPTS": "--driver-memory 4g --executor-memory 4g ..."


In [1]:

var pathToSmallDataset = 'examples/data/mllib/ml-latest-small';
var pathToCompleteDataset = 'examples/data/mllib/ml-latest';


In [2]:

    var SparkConf = require('eclairjs/SparkConf');
    var SparkContext = require('eclairjs/SparkContext');
    var sparkConf = new SparkConf().setAppName("JavaScript Movie");
    var sc = new SparkContext(sparkConf);
    run(sc);

    sc.stop();
